In [1]:
import os
os.chdir("../")

In [2]:
%pwd

'/home/eman/Desktop/DataScience_proj'

In [4]:
import os

In [ ]:
os.environ[MODEL_TRACKING_URI]

In [7]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_filename: Path
    TARGET_COLUMN: str
    #mlflow_uri: str


In [8]:
from src.Datascience_proj.utils.common import read_yaml, create_directories, save_json
from src.Datascience_proj.constants import *

In [10]:
class ConfigurationManager:
    def __init__(self, 
                config_filepath=CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH,
                schema_filepath = SCHEMA_FILE_PATH):
        
        ## read the file and assign to variable
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        ##creating our artifacts directory
        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self)-> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params= self.params.ElasticNet
        schema= self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config= ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            all_params = params,
            metric_filename = config.metric_filename,
            TARGET_COLUMN = schema.name
        )

        return model_evaluation_config

In [11]:
## importing the needed libraries
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [18]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)

        return rmse, mae, r2

    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_X = test_data.drop([self.config.TARGET_COLUMN], axis =1)
        test_y = test_data[self.config.TARGET_COLUMN]

        #mlflow.set_registry_uri(self.config.mlflow_uri)
        #tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(test_X)

            (rmse,mae, r2) = self.eval_metrics(test_y, predicted_qualities)

            ## save the metrics as local
            scores ={'rmse': rmse, 'mae': mae, 'r2': r2}
            save_json(path=Path(self.config.metric_filename), data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric('mae', mae)

            # Model Registry does not work with file store
            #if tracking_uri_type_store != 'file':

                # Register te model
                # Ther are other ways to use the Model Registry, wich depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
               # mlflow.sklearn.log_model(model, 'model', registered_model_name = "ElasticNetModel")
            
            #else:
            mlflow.sklearn.log_model(model, 'model')

In [19]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2025-07-17 07:07:04,183: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-07-17 07:07:04,187: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-17 07:07:04,192: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-07-17 07:07:04,194: INFO: common: created directory at: artifacts]
[2025-07-17 07:07:04,195: INFO: common: created directory at: artifacts/model_evaluation]


[2025-07-17 07:07:04,252: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]


2025/07/17 07:07:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
